In [1]:
import pandas as pd
import google.generativeai as palm
import config
import re

/Users/karthickmani/Documents/Python/youtube_recipes_llm/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('../data/recipes.csv')
palm.configure(api_key=config.palm_api_key)
small_df = df.head(5)

In [3]:
def palm_gen_recipe(transcript, index):
    if index%10 == 0:
        print("Processing record {}".format(index))

    
    defaults = {
      'model': 'models/text-bison-001',
      'temperature': 0.7,
      'candidate_count': 1,
      'top_k': 40,
      'top_p': 0.95,
      'max_output_tokens': 1500,
      'stop_sequences': [],
    }
    prompt = """
    You are assuming the role of a renowned chef who is sharing his recipes using captions from a video. Your task is to perform the following actions:
    1 - Summarize the following text into list of ingredients and step by step recipe instructions: {}
    2 - Output a json object that contains the following keys: ingredients, recipe

    Use the following format, and separate each section with two new lines:
    ===Ingredients===: numbered list of <ingredients> - <quantity> - <unit of measurement>
    ===Recipe===: step by step recipe with each step numbered
    ===Output JSON===: <json with Ingredients, Recipe>

    Please make sure you seperate each section with line breakers '==='
    """.format(transcript)
    try:

        response = palm.generate_text(
          **defaults,
          prompt=prompt
        )
        return response.result
    except:
        return "Not Supported"

In [4]:
df['ingredients_recipe'] = df.apply(lambda x: palm_gen_recipe(x.transcript, x.name), axis=1)
df['ingredients'] = df['ingredients_recipe'].str.extract(r'===Ingredients===\n(.*)===Recipe===', re.DOTALL)
df['recipe'] = df['ingredients_recipe'].str.extract(r'===Recipe===\n(.*)===Output JSON===', re.DOTALL)
df['outputJSON'] = df['ingredients_recipe'].str.extract(r'===Output JSON===\n(.*)', re.DOTALL)

Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0
Processing record 0


In [5]:
df.to_csv('../data/recipe_with_instructions.csv', index=False)


### Test Section

In [6]:
# print(palm_gen_recipe(df['transcript'][0], 0).split('===Output JSON===')[0])